In [19]:
import pandas as pd
import numpy as np
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/'
'machine-learning-databases/wine/wine.data',
header=None)
df_wine.columns = ['Class label', 'Alcohol','Malic acid', 'Ash','Alcalinity of ash', 'Magnesium','Total phenols', 'Flavanoids','Nonflavanoid phenols','Proanthocyanins','Color intensity', 'Hue','OD280/OD315 of diluted wines','Proline']
print('Class labels', np.unique(df_wine['Class label']))
from sklearn.model_selection import train_test_split
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values
X_train, X_test, y_train, y_test =train_test_split(X, y,test_size=0.3,random_state=0,stratify=y)
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

Class labels [1 2 3]


In [28]:
from sklearn.base import clone
from itertools import combinations
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
class SBS():
    def __init__(self, estimator, k_features,
        scoring=accuracy_score,
        test_size=0.25, random_state=1):
        self.scoring = scoring
        self.estimator = clone(estimator)
        self.k_features = k_features
        self.test_size = test_size
        self.random_state = random_state

    def fit(self, X, y):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=self.test_size,
        random_state=self.random_state)
        dim = X_train.shape[1]
        self.indices_ = tuple(range(dim))
        self.subsets_ = [self.indices_]
        score = self._calc_score(X_train, y_train,
        X_test, y_test, self.indices_)
        self.scores_ = [score]
        while dim > self.k_features:
            scores = []
            subsets = []
            for p in combinations(self.indices_, r=dim - 1):
                score = self._calc_score(X_train, y_train,
                X_test, y_test, p)
                scores.append(score)
                subsets.append(p)
                best = np.argmax(scores)
                self.indices_ = subsets[best]
            print("subsets s formed after dim : "+str(dim))
            print(subsets)
            print("best subsets are for dim :" +str(dim))
            self.subsets_.append(self.indices_)
            print(self.indices_)
            dim -= 1
            self.scores_.append(scores[best])
        self.k_score_ = self.scores_[-1]
        print(self.subsets_)
        return self
    
    def transform(self, X):
        return X[:, self.indices_]
    def _calc_score(self, X_train, y_train, X_test, y_test,indices):
        self.estimator.fit(X_train[:, indices], y_train)
        y_pred = self.estimator.predict(X_test[:, indices])
        score = self.scoring(y_test, y_pred)
        return score

In [29]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
sbs = SBS(knn, k_features=1)
sbs.fit(X_train_std, y_train)

subsets s formed after dim : 13
[(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11), (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12), (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12), (0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12), (0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12), (0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12), (0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12), (0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12), (0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12), (0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12), (0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), (0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)]
best subsets are for dim :13
(0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12)
subsets s formed after dim : 12
[(0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11), (0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 12), (0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 12), (0, 1, 2, 3, 4, 5, 6, 7, 10, 11, 12), (0, 1, 2, 3, 4, 5, 6, 9, 10, 11, 12), (0, 1, 2, 3, 4, 5, 7, 9, 10, 11, 12), (0, 1, 2, 3, 4, 6, 7, 9, 10, 11, 12), (0, 1, 2, 3, 5, 6, 7, 9, 10, 11, 12), (0, 1, 2, 4, 5, 6, 7, 9, 10, 11, 1

In [26]:
sbs.k_score_

0.7741935483870968

In [27]:
sbs.scores_

[0.967741935483871,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.967741935483871,
 0.967741935483871,
 0.967741935483871,
 1.0,
 0.9354838709677419,
 0.7741935483870968]

In [16]:
sbs.subsets_[10]

(0, 1, 11)

In [17]:
k3 = list(sbs.subsets_[10])
print(df_wine.columns[1:][k3])

Index(['Alcohol', 'Malic acid', 'OD280/OD315 of diluted wines'], dtype='object')


In [18]:
knn.fit(X_train_std, y_train)
print('Training accuracy:', knn.score(X_train_std, y_train))
print('Test accuracy:', knn.score(X_test_std, y_test))

Training accuracy: 0.967741935483871
Test accuracy: 0.9629629629629629
